In [1]:
import pymysql
import requests
import pandas as pd
import numpy as np

### Genera un df con el archivo `csv` "CPTEVENTS"

In [2]:
df = pd.read_csv('C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\CPTEVENTS.csv', encoding='UTF-8', delimiter=',', decimal='.')
df.head(3)

,row_id,subject_id,hadm_id,costcenter,chartdate,cpt_cd,cpt_number,cpt_suffix,ticket_id_seq,sectionheader,subsectionheader,description
0,4615,10117,105150,ICU,NaN,99254,99254,NaN,1.0,Evaluation and management,Consultations,NaN
1,4616,10117,105150,ICU,NaN,99231,99231,NaN,2.0,Evaluation and management,Hospital inpatient services,NaN
2,4617,10117,105150,ICU,NaN,90935,90935,NaN,3.0,Medicine,Dialysis,NaN


In [3]:
df = df.replace(np.nan, 0)

### Se conecta con la BD MySQL AWS

In [4]:
con = pymysql.connect(
      host='database-proyecto-final.cpjrl88bhdul.us-east-1.rds.amazonaws.com',
      database='databaseuci',
      user='admin',
      password='datascience'
)

### Inserta los registros del df "CPTEVENTS" en la BD

In [5]:
sql_insert = ''' INSERT 
                   INTO  CPTEVENTS
                         (row_id, subject_id, hadm_id, costcenter, chartdate, cpt_cd, cpt_number, cpt_suffix, ticket_id_seq, 
                          sectionheader, subsectionheader, description)
                 VALUES  (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) '''
num_reng = 0
cursor = con.cursor()
for i in range(0, len(df)):
    row = int(df.loc[i, 'row_id'])
    subj = int(df.loc[i, 'subject_id'])
    hadm = int(df.loc[i, 'hadm_id'])
    coce = df.loc[i, 'costcenter']
    chdt = df.loc[i, 'chartdate']
    cptc = df.loc[i, 'cpt_cd']
    cptn = int(df.loc[i, 'cpt_number'])
    cpts = df.loc[i, 'cpt_suffix']
    tkid = int(df.loc[i, 'ticket_id_seq'])
    sehd = df.loc[i, 'sectionheader']
    sbhd = df.loc[i, 'subsectionheader']
    desc = df.loc[i, 'description']
    reg_s = (row, subj, hadm, coce, chdt, cptc, cptn, cpts, tkid, sehd, sbhd, desc)
    cursor.execute(sql_insert, reg_s)
    num_reng += 1
    if num_reng == 100:
        con.commit()
        num_reng = 0

### Confirma la inserción de los registros a la BD

In [6]:
con.commit()

OperationalError: (2013, 'Lost connection to MySQL server during query ([WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder)')

### Cierra el cursor y la conexion a la BD

In [7]:
cursor.close()
con.close()